In [1]:
import json
import cv2
import os

In [2]:
# creating annotations
annotations = []
images = []
categories = [{"id": 1, "name": "barnacle"}]

# iterating through all the images and masks to create annotations
for idx, img_path in enumerate(os.listdir("barnacle_dataset/train/images")):
    # Add image entry
    img = cv2.imread(f"barnacle_dataset/train/images/{img_path}")
    if img is None:
        print(f"Warning: Unable to read image file {img_path}")
        continue

    images.append({
        "id": idx,
        "file_name": img_path,
        "width": img.shape[1],
        "height": img.shape[0],
    })

    # Construct mask file path
    mask_path = f"barnacle_dataset/train/masks/mask_{img_path}"
    if not os.path.exists(mask_path):
        print(f"Warning: Mask file {mask_path} does not exist")
        continue

    # Convert mask to polygons
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Warning: Unable to read mask file {mask_path}")
        continue

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        annotations.append({
            "id": len(annotations),
            "image_id": idx,
            "category_id": 1,
            "segmentation": [cnt.flatten().tolist()],
            "bbox": cv2.boundingRect(cnt),
            "area": cv2.contourArea(cnt),
            "iscrowd": 0,
        })

In [3]:
# Save annotations
with open("barnacle_dataset/annotations/train.json", "w") as f:
    json.dump({"images": images, "annotations": annotations, "categories": categories}, f)